<a href="https://colab.research.google.com/github/elucidator8918/SIH-2023/blob/ML/SIH_TTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install selenium TTS fastapi[all] pyngrok nest_asyncio sentencepiece twilio elevenlabs -q
!apt-get update
!apt install -y chromium-chromedriver
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok
!ngrok authtoken 2NqtkyBMF0KF99ofEU1fGz1pCJS_3XehxpoU47JGBiqY6tV7M

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 938.0/938.0 kB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 461.6/461.6 kB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 63.6 MB/s e

In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [3]:
!git config --global user.name "<YOUR USER NAME>"
!git config --global user.email "YOUR EMAIL ID"
!git config --global user.password "<PASSWORD>"

token = '<GITHUB PSA>'
username = '<GITHUB USERNAME>'
repo = '<REPO>'
!git clone https://{token}@github.com/{username}/{repo}
%cd {repo}

Cloning into 'xml_vercel'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 59 (delta 4), reused 36 (delta 0), pack-reused 0
Receiving objects: 100% (59/59), 4.76 MiB | 1.72 MiB/s, done.
Resolving deltas: 100% (4/4), done.
/content/xml_vercel


In [4]:
from TTS.api import TTS
import requests
from fastapi import FastAPI, UploadFile, File, HTTPException, Body, Depends, HTTPException, status, Header
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
from fastapi.encoders import jsonable_encoder
from fastapi.responses import JSONResponse
import uuid
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import torch
from transformers import AutoTokenizer, VitsModel
from pydantic import BaseModel
import nest_asyncio
from twilio.rest import Client
import os
import asyncio
from fastapi.responses import FileResponse
import uvicorn
from fastapi.responses import JSONResponse
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import soundfile as sf
from selenium.webdriver.support.ui import WebDriverWait
import time
import re
from typing import List
import numpy as np
from elevenlabs import generate, play, set_api_key, save

set_api_key("<API KEY>")

device = torch.cuda.current_device() if torch.cuda.is_available() else 'cpu'
tts = TTS('tts_models/en/jenny/jenny').to(device)
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

tam_model = VitsModel.from_pretrained("facebook/mms-tts-tam")
tam_tokenizer = AutoTokenizer.from_pretrained("facebook/mms-tts-tam")

guj_model = VitsModel.from_pretrained("facebook/mms-tts-guj")
guj_tokenizer = AutoTokenizer.from_pretrained("facebook/mms-tts-guj")
port = 8888

# account_sid = "AC1966edb3d61979843cd38183dfcf8cb0"
# auth_token = "b33d7f9292aea9e8c78cd034eb7bf6ed"
# twilio_number = '+12059973367'

account_sid = '<SECRET>'
auth_token = 'SECRET'
twilio_number = ''

chrome_options = Options()
chrome_options.add_argument('--headless')  # Optional: Run Chrome in headless mode (no GUI)
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

chrome_options.binary_location = '/usr/bin/google-chrome'

class TTSNER(BaseModel):
    text: str
    emotion: str = "Cheerful & Professional"

class TRANS(BaseModel):
    text: str
    src_lang: str
    tgt_lang: str

class TrainData1(BaseModel):
    train_name: List[str]
    train_no: List[str]
    total_votes: List[str]
    cleanliness: List[str]
    punctuality: List[str]
    food: List[str]
    ticket: List[str]
    safety: List[str]

class TrainData2(BaseModel):
    Type: List[str]
    Zone: List[str]
    From: List[str]
    PFfrom: List[str]
    Dep: List[str]
    AvgDelay: List[str]
    To: List[str]
    PFto: List[str]
    Arr: List[str]

class INFO(BaseModel):
    station_1: str
    station_2: str

class TrainSchedule(BaseModel):
    train_name: str
    platform_number: str
    departure_time: str
    train_no: str

ngrok_tunnel = ngrok.connect(port)
print('Public URL:', ngrok_tunnel.public_url)

 > Downloading model to /root/.local/share/tts/tts_models--en--jenny--jenny


100%|██████████| 1.61G/1.61G [00:49<00:00, 84.4MiB/s]

 > Model's license - custom - see https://github.com/dioco-group/jenny-tts-dataset#important
 > Check https://opensource.org/licenses for more info.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:48000
 | > resample:False
 | > num_mels:100
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:2048
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:512
 | > win_length:2048


config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

...

Some weights of the model checkpoint at facebook/mms-tts-guj were not used when initializing VitsModel: ['flow.flows.2.wavenet.res_skip_layers.0.weight_g', 'flow.flows.0.wavenet.res_skip_layers.2.weight_v', 'posterior_encoder.wavenet.res_skip_layers.3.weight_g', 'flow.flows.1.wavenet.res_skip_layers.0.weight_v', 'flow.flows.1.wavenet.in_layers.3.weight_v', 'posterior_encoder.wavenet.in_layers.7.weight_g', 'flow.flows.0.wavenet.res_skip_layers.3.weight_v', 'flow.flows.1.wavenet.res_skip_layers.1.weight_v', 'posterior_encoder.wavenet.res_skip_layers.12.weight_v', 'posterior_encoder.wavenet.in_layers.4.weight_g', 'posterior_encoder.wavenet.res_skip_layers.7.weight_g', 'posterior_encoder.wavenet.res_skip_layers.14.weight_v', 'posterior_encoder.wavenet.in_layers.14.weight_g', 'posterior_encoder.wavenet.in_layers.0.weight_v', 'flow.flows.0.wavenet.res_skip_layers.0.weight_g', 'flow.flows.0.wavenet.in_layers.1.weight_v', 'flow.flows.2.wavenet.res_skip_layers.1.weight_g', 'posterior_encoder.wa

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/765 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/47.0 [00:00<?, ?B/s]

Public URL: https://0f34-34-143-168-214.ngrok-free.app


In [5]:
chrome_options = Options()
chrome_options.add_argument('--headless')  # Optional: Run Chrome in headless mode (no GUI)
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

chrome_options.binary_location = '/usr/bin/google-chrome'

def extract_text_and_links(html_string):
    clean_text = []

    train_data1 = {
        "train_name": [],
        "train_no": [],
        "total_votes": [],
        "cleanliness": [],
        "punctuality": [],
        "food": [],
        "ticket": [],
        "safety": [],
    }

    train_data2 = {
        "Type": [],
        "Zone": [],
        "From": [],
        "PFfrom": [],
        "Dep": [],
        "AvgDelay": [],
        "To": [],
        "PFto": [],
        "Arr": [],
    }

    # Find all occurrences of text inside <div> tags
    matches = re.findall(r'<div.*?>(.*?)</div>', html_string, re.DOTALL)

    # Append each match to the clean_text list
    clean_text.extend(matches)
    # Remove unnecessary elements
    clean_text = clean_text[20:]

    name_pattern = r'title1="([^"]*)"'
    lengths = []
    votes_pattern = r'\((\d+) votes\)'
    votes_patter2 = r'\((\d+)\)'
    for i in range(0, len(clean_text)):
        if clean_text[i] == '':
            lengths.append(i)
    j=0
    while j<len(lengths) and j<5:  # Adjusted the loop range
        the_rest = []  # Moved inside the loop to reset for each segment
        for i in range(lengths[j], lengths[j + 1], 1):
            if "title1" in clean_text[i]:
                train_data1["train_name"].append(re.search(name_pattern, clean_text[i]).group(1))
                train_data1["train_no"].append(clean_text[i][-5:])
            elif "votes" in clean_text[i]:
                train_data1["total_votes"].append(re.search(votes_pattern,clean_text[i]).group(1))
            elif "cleanliness" in clean_text[i]:
                train_data1["cleanliness"].append(re.search(votes_patter2,clean_text[i]).group(1))
            elif "punctuality" in clean_text[i]:
                train_data1["punctuality"].append(re.search(votes_patter2,clean_text[i]).group(1))
            elif "food" in clean_text[i]:
                train_data1["food"].append(re.search(votes_patter2,clean_text[i]).group(1))
            elif "ticket" in clean_text[i]:
                train_data1["ticket"].append(re.search(votes_patter2,clean_text[i]).group(1))
            elif "safety" in clean_text[i]:
                train_data1["safety"].append(re.search(votes_patter2,clean_text[i]).group(1))
            else:
                if len(clean_text[i]) <= 6:
                    the_rest.append(clean_text[i])
        j += 1

        # Check if the_rest list has enough elements before accessing them
        if len(the_rest) >= len(train_data2):
            k = 1
            for keys in train_data2.keys():
                train_data2[keys].append(the_rest[k])
                k += 1
    return train_data1, train_data2


def perform_search_util(station_1, station_2):

    driver = webdriver.Chrome(options=chrome_options)
    driver.get("https://indiarailinfo.com/")
    from_station = driver.find_element(By.XPATH, '//input[@placeholder="from station"]')
    from_station.clear()
    from_station.send_keys(station_1 + Keys.DOWN + Keys.ENTER)
    time.sleep(2)
    driver.find_element(By.XPATH, '//div[@class="list showslow"]/table/tbody/tr').click()



    to_station = driver.find_element(By.XPATH, '//input[@placeholder="to station"]')
    to_station.clear()
    to_station.send_keys(station_2 + Keys.DOWN+ Keys.ENTER)
    time.sleep(1)
    driver.find_element(By.XPATH, '//div[@class="list showslow"]').click()

    # Get the HTML content of the div with class "srhres newbg inline alt"
    driver.find_element(By.XPATH, '//div[@id="SrhDiv"]').click()
    try:
        div_content = driver.find_element(By.XPATH, "//div[@class ='srhres newbg inline alt']").get_attribute("outerHTML")
    except:
        return "No Trains Found"
    trian_data1,train_data2 = extract_text_and_links(div_content)
    driver.quit()
    return trian_data1, train_data2

In [6]:
def translate(text,src_lang,tgt_lang):
    tokenizer.src_lang = src_lang
    encoded_text = tokenizer(text, return_tensors="pt")
    src_lang_code = tokenizer.lang_code_to_id[src_lang]
    tgt_lang_code = tokenizer.lang_code_to_id[tgt_lang]
    generated_tokens = model.generate(
        **encoded_text,
        forced_bos_token_id=tgt_lang_code,
    )
    translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    return translated_text

In [ ]:
def enhance_translation(original_text, translated_text):
    term_translations = {
        'platform': 'பரதி',
        'train': 'ரயில்',
        'station': 'நிலையம்',
        'arrival': 'அனுப்புதல்',
        'delayed': 'தாமதமாகிவிட்டது',
        'cancelled': 'ரத்துசெய்யப்பட்டது',
        'schedule': 'அட்டவணை',
        'passenger': 'பயணிகள்',
        'ticket': 'டிக்கெட்',
        'announcement': 'அறிவிப்பு',
        'emergency': 'அவசரச்',
        'information': 'தகவல்',
        'platform_number': 'பரதி எண்',
        'connecting_train': 'இணைக்குநர் ரயில்',
        'cancellation': 'ரத்து'
    }


    original_words = original_text.lower().split()
    translated_words = translated_text.lower().split()


    for term in term_translations:
        if term not in translated_words:
            index_in_original = original_words.index(term) if term in original_words else None

            if index_in_original is not None and index_in_original < len(translated_words):
                translated_words.insert(index_in_original, term_translations[term])
            else:
                translated_words.append(term_translations[term])

    # Join the words back into a sentence
    enhanced_translation = ' '.join(translated_words)
    return enhanced_translation

In [14]:
app = FastAPI(title="SIH 2023 Backend")

app.add_middleware(
CORSMiddleware,
allow_origins=["*"],
allow_credentials=True,
allow_methods=["*"],
allow_headers=["*"],
)

@app.get("/")
async def home():
    return "SIH - LICHT DEN CODE"

@app.post("/coqui-tts/")
async def coqui_tts(request: TTSNER = Body(...)):#,token: str = Header(...)):
    try:
      out = f"{uuid.uuid4()}.ogg"
      async def remove():
          loop = asyncio.get_event_loop()
          await loop.run_in_executor(None, lambda: os.remove(out))
      tts.tts_to_file(request.text, file_path=out, emotion=request.emotion)
      return FileResponse(out,headers={"Content-Disposition":f"attachment; filename={out}"},background=remove)
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/tamil-tts/")
async def tamil_tts(request: TTSNER = Body(...)):#,token: str = Header(...)):
    try:
      #decoded_token = decode_token(token)
      out = f"{uuid.uuid4()}.ogg"
      async def remove():
          loop = asyncio.get_event_loop()
          await loop.run_in_executor(None, lambda: os.remove(out))
      inputs = tam_tokenizer(request.text, return_tensors="pt")
      with torch.no_grad():
        output = tam_model(**inputs).waveform
      sf.write(out, np.ravel(output), tam_model.config.sampling_rate)
      return FileResponse(out,headers={"Content-Disposition":f"attachment; filename={out}"},background=remove)
    except HTTPException as e:
        raise e
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/gujarati-tts/")
async def gujarati_tts(request: TTSNER = Body(...)):#,token: str = Header(...)):
    try:
      #decoded_token = decode_token(token)
      out = f"{uuid.uuid4()}.ogg"
      async def remove():
          loop = asyncio.get_event_loop()
          await loop.run_in_executor(None, lambda: os.remove(out))
      inputs = guj_tokenizer(request.text, return_tensors="pt")
      with torch.no_grad():
        output = guj_model(**inputs).waveform
      sf.write(out, np.ravel(output), guj_model.config.sampling_rate)
      return FileResponse(out,headers={"Content-Disposition":f"attachment; filename={out}"},background=remove)
    except HTTPException as e:
        raise e
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/translate")
async def translate_text(request: TRANS):
    tokenizer.src_lang = request.src_lang
    encoded_text = tokenizer(request.text, return_tensors="pt")
    src_lang_code = tokenizer.lang_code_to_id[request.src_lang]
    tgt_lang_code = tokenizer.lang_code_to_id[request.tgt_lang]
    generated_tokens = model.generate(
        **encoded_text,
        forced_bos_token_id=tgt_lang_code,
    )
    translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    return {"translated_text": translated_text}

@app.post("/gtranslate")
async def gtranslate_text(request: TRANS):

    url = "https://google-translate1.p.rapidapi.com/language/translate/v2"
    headers = {
        "content-type": "application/x-www-form-urlencoded",
        "Accept-Encoding": "application/gzip",
        "X-RapidAPI-Key": "key",
        "X-RapidAPI-Host": "google-translate1.p.rapidapi.com"
    }
    payload = {
          "q": f"{request.text}",
          "target": f"{request.tgt_lang}",
          "source": f"{request.src_lang}"
        }

    response = requests.post(url, data=payload, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        raise HTTPException(status_code=response.status_code, detail="Translation failed")

@app.post("/get_train_data")
async def perform_search(request: INFO):
    # Call your existing function to get train data
    train_data1, train_data2 = perform_search_util(request.station_1, request.station_2)

    # Create instances of the Pydantic models
    response_train_data1 = TrainData1(**train_data1)
    response_train_data2 = TrainData2(**train_data2)

    # Return the data as JSON
    data = {"train_data1": jsonable_encoder(response_train_data1), "train_data2": jsonable_encoder(response_train_data2)}
    return JSONResponse(content=data)

@app.post("/make_call")
async def make_call(request: TrainSchedule):
    client = Client(account_sid, auth_token)
    twiml = f'''<Response><Say>
                Welcome to Railway Mitron! We kindly inform you that the
                {request.train_name} - {request.train_no} train will be departing from Platform {request.platform_number} at {request.departure_time}. Thank you!
                </Say></Response>'''

    lst = ["+919820086903","+918850014291","+918452899222","+919833676226"]

    for number in lst:
      call = client.calls.create(
        twiml=twiml,
        to=number,
        from_=twilio_number)
      call.sid

    # TAMIL
    twiml = '''<Response><Say>Welcome to Railway Mitra! </Say> <Play>https://xml-vercel.vercel.app/classic.mp3</Play> </Response>'''
    text = f'''We kindly inform you that the {request.train_name} - {request.train_no} train will be departing from Platform {request.platform_number} at {request.departure_time}. Thank you! '''
    out = "classic.mp3"
    text = translate(text,"en_XX","ta_IN")
    # inputs = tam_tokenizer(text, return_tensors="pt")
    # with torch.no_grad():
    #   output = tam_model(**inputs).waveform
    # sf.write(out, np.ravel(output), tam_model.config.sampling_rate)
    audio = generate(
        text=text,
        voice="Paul",
        model="eleven_multilingual_v2"
      )
    save(audio,out)

    !git add classic.mp3
    !git commit -a -m "Added a SOMETHING BRO"
    !git remote -v
    !git push origin main
    time.sleep(15)
    call = client.calls.create(twiml=twiml,to="num",from_=twilio_number)
    call.sid

    # Hindi

    twiml = '''<Response><Say>Welcome to Railway Mitra! </Say> <Play>https://xml-vercel.vercel.app/classic.mp3</Play> </Response>'''
    text = f'''We kindly inform you that the {request.train_name} - {request.train_no} train will be departing from Platform {request.platform_number} at {request.departure_time}. Thank you! '''
    out = "classic.mp3"
    text = translate(text,"en_XX","hi_IN")

    # inputs = guj_tokenizer(text, return_tensors="pt")
    # with torch.no_grad():
    #   output = guj_model(**inputs).waveform
    # sf.write(out, np.ravel(output), guj_model.config.sampling_rate)
    audio = generate(
        text=text,
        voice="Paul",
        model="eleven_multilingual_v2"
      )
    save(audio,out)

    !git add classic.mp3
    !git commit -a -m "Added a SOMETHING BRO"
    !git remote -v
    !git push origin main
    time.sleep(15)
    call = client.calls.create(twiml=twiml,to="num",from_=twilio_number)
    call.sid

    return {"status": "Call initiated"}

@app.post("/make_SMS")
async def make_sms(request: TrainSchedule):
    client = Client(account_sid, auth_token)
    twiml = f'''Welcome to Railway Mitron! We kindly inform you that the
                {request.train_name} - {request.train_no} train will be departing from Platform {request.platform_number} at {request.departure_time}. Thank you!
                '''

    lst = []

    for number in lst:
      message = client.messages.create(body=twiml,
        to=number,
        from_=twilio_number)
      message.sid

    tamtwiml = translate(twiml,"en_XX","ta_IN")
    gutwiml = translate(twiml,"en_XX","hi_IN")

    message = client.messages.create(body=tamtwiml,to="+919819971058",from_=twilio_number)
    message.sid

    message = client.messages.create(body=gutwiml,to="+919967019397",from_=twilio_number)
    message.sid

    return {"status": "SMS initiated"}

if __name__=="__main__":
    nest_asyncio.apply()
    uvicorn.run(app,port=port)

INFO:     Started server process [744]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8888 (Press CTRL+C to quit)


INFO:     121.200.55.226:0 - "GET / HTTP/1.1" 200 OK
INFO:     121.200.55.226:0 - "OPTIONS /gtranslate HTTP/1.1" 200 OK
INFO:     121.200.55.226:0 - "POST /gtranslate HTTP/1.1" 400 Bad Request
INFO:     121.200.55.226:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     121.200.55.226:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     121.200.55.226:0 - "POST /gtranslate HTTP/1.1" 200 OK
INFO:     121.200.55.226:0 - "POST /gtranslate HTTP/1.1" 400 Bad Request
INFO:     121.200.55.226:0 - "POST /gtranslate HTTP/1.1" 200 OK
INFO:     121.200.55.226:0 - "OPTIONS /gtranslate HTTP/1.1" 200 OK
INFO:     121.200.55.226:0 - "POST /gtranslate HTTP/1.1" 200 OK
INFO:     121.200.55.226:0 - "OPTIONS /get_train_data HTTP/1.1" 200 OK
INFO:     121.200.55.226:0 - "POST /get_train_data HTTP/1.1" 200 OK
INFO:     121.200.55.226:0 - "GET / HTTP/1.1" 200 OK
INFO:     121.200.55.226:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     121.200.55.226:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     121.200.55.226:0 - "GET

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [744]


In [13]:
ngrok_tunnel = ngrok.connect(port)
print('Public URL:', ngrok_tunnel.public_url)

Public URL: https://f248-34-143-168-214.ngrok-free.app
